# 🌊 GradTracer v0.7.0 — Complete Demo

**Training Diagnostics, Standard JSON Export, W&B Bridge, Mixed Precision Auto-Compression Recipe from Training Dynamics.**

Run all cells in order.

In [ ]:
# Install GradTracer (Colab / fresh env)
# !pip install gradtracer
# !pip install torch xgboost lightgbm scikit-learn statsmodels

In [ ]:
import gradtracer
gradtracer.info()

---
## Section 1: PyTorch FlowTracker — DL Training Diagnostics

In [ ]:
import torch
import torch.nn as nn
from gradtracer import FlowTracker

# Simple model
model = nn.Sequential(
    nn.Linear(20, 64),
    nn.BatchNorm1d(64),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 1),
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

tracker = FlowTracker(
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    run_name="exp_01_baseline"
)

# Simulate training
X = torch.randn(128, 20)
y = torch.randn(128, 1)

for epoch in range(50):
    pred = model(X)
    loss = nn.MSELoss()(pred, y)
    loss.backward()
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()
    tracker.step(loss=loss.item())

tracker.report()

---
## Section 2: AI Agent Mode — Standardized JSON Exporter

In [ ]:
# Export training context as Standard JSON for modern AI coders
from gradtracer.agent import AgentExporter
json_report = AgentExporter.export_dl(tracker, save=False)
print(json_report)

---
## Section 3: W&B Bridge (Weights & Biases Integration)

In [ ]:
from gradtracer.wandb import log_to_wandb

# Import wandb in your training loop normally:
# import wandb
# wandb.init(project="gradtracer_demo")

# Then inside the loop, just call one line:
# log_to_wandb(tracker, step=50)

print("✅ Try integrating W&B by uncommenting the lines above.")

---
## Section 4: Auto-Compression Recipe (`RecipeGenerator`)
Instead of randomly pruning, give the tracking data directly to the RecipeGenerator, which balances SNR and Velocity to prescribe INT4 / INT8 or Structural Pruning ratios.

In [ ]:
from gradtracer.analyzers.recipes import RecipeGenerator
import json

recipe_gen = RecipeGenerator(tracker.store)
best_recipe = recipe_gen.generate(target_sparsity=0.5)

print(json.dumps(best_recipe, indent=2))

# This exports for the VS Code Extension:
# recipe_gen.export_json("gradtracer_recipe.json")

---
## Section 5: Feature Engineering & VIF Diagnostics (Classic)

In [ ]:
from gradtracer import FeatureAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
import numpy as np

X_cls, y_cls = make_classification(n_samples=500, n_features=10, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_cls, y_cls)

feature_names = [f'feature_{i}' for i in range(10)]
analyzer = FeatureAnalyzer(rf, X_cls, y_cls, feature_names=feature_names)

# Suggested Interactions
interactions = analyzer.interactions(top_k=5)
print("Top interactions:")
for item in interactions:
    print(f"  {item['feat_a']} × {item['feat_b']}: synergy={item['synergy_score']:.4f}")